<center>
<h style="line-height: 0.5;">

# Applying Machine Learning to Gravitational Lens Modeling
#### <ins>Research Advisor</ins>: Professor Charles Keeton
#### <ins>Student</ins>: Satyajit Gade

</h>
</center>

<h style="line-height: 0.5;">

##### <ins>Assumptions for Lens Galaxy</ins>:

<p style="line-height: 1;">

*Redshift*: uniform $0.2 < z_l < 0.5$\
*Einstein Radius*: uniform $1.0 < \theta_E < 1.5$ arcsecs\
*Power law exponent*: Gaussian with  mean and standard deviation $\eta = 1.0 \pm 0.1$\
*Ellipticity*: uniform $0 < e < 0.5$ with uniform random angle

</p>

##### <ins>Assumptions for Line of Sight</ins>:

<p style="line-height: 1;">

*External Shear*: uniform $0 < \gamma < 0.1$ with uniform random angle\
*External Convergence*: none

</p>

##### <ins>Assumptions for Source</ins>:

<p style="line-height: 1;">

*Redshift*: uniform $1 < z_s < 3$\
*Position*: uniform density 0.75 per square arcsec

</p>
</h>

In [8]:
# imports all the modules needed

import math
import numpy as np
import matplotlib.pyplot as plt
import pygravlens as gl
from astropy.cosmology import Planck18 as cosmo

### Base Params and Inputs

In [9]:
# Generating Mock Lenses for Training Data

# number of mock lenses
num_mock = 3000

In [10]:
# generates random einstein radii 
EinsArr = np.random.uniform(1.0, 1.5, num_mock) 

# from a radius of 0 to 1 in polar coords
r = np.sqrt(np.random.uniform(0.0, 1.0, num_mock))

theta_src = np.random.uniform(0, 2*math.pi, num_mock)

# random source positions
betaOne = [];
betaTwo = [];

for i in range(num_mock):
    # x = rcos theta
    # y = rsin theta
    betaOne.append(r[i]*np.cos(theta_src[i]))
    betaTwo.append(r[i]*np.sin(theta_src[i]))

# randomized shear vals b/w 0 and 0.1 and ellipticity vals between 0 and 0.5
shear_vals = np.random.uniform(0, 0.1, num_mock)
ellip_vals = np.random.uniform(0, 0.5, num_mock)

# random theta vals

theta_e = np.random.uniform(0, 2*math.pi, num_mock)
theta_g = np.random.uniform(0, 2*math.pi, num_mock)

# ec, es; gc, gs
'''
ec = ellip*cos(2*theta_e)
es = ellip*sin(2*theta_e)
gc = gamma*cos(2*theta_g)
gs = gamma*sin(2*theta_g)
'''
ec = [];
es = [];
gc = [];
gs = [];

for i in range(num_mock):
    ec.append(ellip_vals[i]*np.cos(2*theta_e[i]))
    es.append(ellip_vals[i]*np.sin(2*theta_e[i]))
    gc.append(shear_vals[i]*np.cos(2*theta_g[i]))
    gs.append(shear_vals[i]*np.sin(2*theta_g[i]))

# accounting for red-shift:
zlens = np.random.uniform(0.2, 0.5, num_mock);
zsrc = np.random.uniform(1.0, 3.0, num_mock);

Dlens = [];
Dsrc = [];

for i in range(num_mock):
    Dlens.append(cosmo.comoving_distance(zlens[i]));
    Dsrc.append(cosmo.comoving_distance(zsrc[i]));

xtmp_elpow = np.random.uniform(low=-2,high=2,size=(1000,2))



### Generate Mock Lens Function:

In [11]:
def Generate_MockLens(args):
    
    shear_only, ellip_only, both = args

    # Dictionary related code:
    keys = range(num_mock) # key pairs created for size of EinsArr = number of mock lenses
    values = [] # both
    
    # for loop for creating a dictionary with an array of all the img, magnificaiton, and source arrays
    for i in range(num_mock):
        
        # use model.findimg and pass it the beta values to find image pos
        # need to find source positions and give the model source positions to find images.
        
        # 3rd param = 1.0 for SIS (power law index)
        if shear_only == True & (ellip_only == False & both == False):
            plane_elpow = gl.lensplane('ellpow',[0.0,0.0,1.0,EinsArr[i],0.0,0.0], gammac=gc[i], gammas=gs[i], Dl=Dlens[i])
            
        elif ellip_only == True & (shear_only == False & both == False):
            plane_elpow = gl.lensplane('ellpow',[0.0,0.0,1.0,EinsArr[i],ec[i],es[i]], Dl=Dlens[i])
            
        elif both == True & (ellip_only == False & shear_only == False):
            plane_elpow = gl.lensplane('ellpow',[0.0,0.0,1.0,EinsArr[i],ec[i],es[i]], gammac=gc[i], gammas=gs[i], Dl=Dlens[i])
            
        else:
            break
        
        # plane_elpow.check(xtmp_elpow)

        model_elpow = gl.lensmodel([plane_elpow],Ds=Dsrc[i])
        model_elpow.tile()
        # model_elpow.plotcrit()
        # model_elpow.plotmag()

        imgarr,muarr,tarr = model_elpow.findimg([betaOne[i],betaTwo[i]])
        
        # boolean mask related code to ignore the small really small terms for magnification.
        boolean_mask = np.fabs(muarr) > (10**(-4)) # if magnification array contains values greater than this, keep, otherwise discard.
        newImg_arr = imgarr[boolean_mask]
        newMag_arr = muarr[boolean_mask]
        newTime_arr = tarr[boolean_mask]
        
        # values is a 2d array containing the img positions, magnifications, and time delay of each einstein radius.
        
        if shear_only == True & (ellip_only == False & both == False):
            elpow_dict = dict(img=newImg_arr, mu=newMag_arr, time=newTime_arr, ellipc = 0.0, ellips = 0.0, gammc = gc[i], gamms = gs[i], 
                      einrad = EinsArr[i], zLens = zlens[i], zSrc= zsrc[i], betaOne = betaOne[i], betaTwo = betaTwo[i])
            
        elif ellip_only == True & (shear_only == False & both == False):
            elpow_dict = dict(img=newImg_arr, mu=newMag_arr, time=newTime_arr, ellipc = ec[i], ellips = es[i], gammc = 0.0, gamms = 0.0, 
                      einrad = EinsArr[i], zLens = zlens[i], zSrc= zsrc[i], betaOne = betaOne[i], betaTwo = betaTwo[i])
            
        elif both == True & (ellip_only == False & shear_only == False):
            elpow_dict = dict(img=newImg_arr, mu=newMag_arr, time=newTime_arr, ellipc = ec[i], ellips = es[i], gammc = gc[i], gamms = gs[i], 
                      einrad = EinsArr[i], zLens = zlens[i], zSrc= zsrc[i], betaOne = betaOne[i], betaTwo = betaTwo[i])
        else:
            break
        
        values.append(elpow_dict)

        # model_elpow.plot(src=[betaOne[i],betaTwo[i]])
    
    if shear_only == True & (ellip_only == False & both == False):
        return values
    elif ellip_only == True & (shear_only == False & both == False):
        return values
    elif both == True & (ellip_only == False & shear_only == False):
        return values
    else:
        return "Error: neither \"shear_only\", \"ellip_only\", or \"both\" parameters are true. Therefore, Mock Lenses cannot be generated."

### Saving the Mock Lenses Generated:

In [12]:
## save code:

vals_shear = Generate_MockLens([True, False, False])
vals_ellip = Generate_MockLens([False, True, False])
vals_both = Generate_MockLens([False, False, True])

np.save('valShear.npy', vals_shear)
np.save('valEllip.npy', vals_ellip)
np.save('valBoth.npy', vals_both)
